In [1]:
from types import UnionType
from typing import Any

from pydantic import BaseModel, Field, create_model
from pydantic_ai import Agent

%load_ext autoreload
%autoreload 2

In [19]:
class model(BaseModel):
    email: "string"


In [20]:
m = model(email="test@test.com")
m.model_dump_json()


PydanticUserError: `model` is not fully defined; you should define `string`, then call `model.model_rebuild()`.

For further information visit https://errors.pydantic.dev/2.10/u/class-not-fully-defined

In [ ]:
DynamicFoobarModel = create_model("DynamicFoobarModel", **{"foo": (str, ...), "bar": (int, 123)})

In [ ]:
class FormField(BaseModel):
    name: str
    type: str = Field(description="The type of the field. For example: string, integer, float, boolean, etc.")
    required: bool = True

    description: str | None = None


class FormSpec(BaseModel):
    title: str
    fields: list[FormField]


def create_dynamic_model(form_spec: FormSpec) -> type[BaseModel]:
    field_types: dict[str, tuple[type | UnionType, Any]] = {}

    for field in form_spec.fields:
        # Map string type names to actual Python types
        type_map = {
            "string": (str, ...),
            "integer": (int, ...),
            "float": (float, ...),
            "boolean": (bool, ...),
        }
        field_type = type_map.get(field.type, (str, ...))

        if field.required:
            field_types[field.name] = field_type
        else:
            field_types[field.name] = (field_type[0] | None, None)

    # Create the model dynamically
    return create_model(form_spec.title, **field_types)


form_designer_agent = Agent(
    model="google-gla:gemini-2.0-flash",
    name="form_designer",
    system_prompt=(
        "You are a form designer. Help the user create a form by collecting field information.\n"
        "For each field, ask whether it's required.\n"
        "Set the name and data type (string, integer, float, boolean, etc.), yourself.\n"
        "Once you have all the information, verify the form with the user.\n"
        "Verifying means also verifying the field names and types.\n"
        "Once the form is verified, return a FormSpec object."
    ),
    result_type=FormSpec | str,  # type: ignore
)

In [2]:
class Form(BaseModel):
    name: str
    age: int
    email: str


agent = Agent(
    model="google-gla:gemini-2.0-flash",
    name="form_filler",
    system_prompt=(
        "You are a form filler.\nKeep asking the user for the information until the form is filled out.\n"
    ),
)

In [3]:
user_prompt = f"Fill out the form: {Form.model_json_schema()}"
print(user_prompt)

Fill out the form: {'properties': {'name': {'title': 'Name', 'type': 'string'}, 'age': {'title': 'Age', 'type': 'integer'}, 'email': {'title': 'Email', 'type': 'string'}}, 'required': ['name', 'age', 'email'], 'title': 'Form', 'type': 'object'}


In [4]:
message_history = None
while True:
    res = await agent.run(user_prompt=user_prompt, result_type=Form | str, message_history=message_history)
    if isinstance(res.data, str):
        user_prompt = input(res.data)
    else:
        break
    message_history = res.all_messages()

In [5]:
res.data

Form(name='hamza', age=29, email='hamza@gmail.com')